# Lab 1 - Parte 2

Primeiro, importamos todas as bibliotecas necessárias para realização da tarefa:

In [4]:
import pandas as pd
import numpy as np
import nltk as nl
from collections import defaultdict
from unicodedata import normalize

Uma breve visualização dos dados:

In [5]:
df = pd.read_csv('estadao_noticias_eleicao.csv')
df = df.replace(np.nan, '', regex=True)
df.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5


O método abaixo retira todos os caracteres especiais e pontuação de uma query:

In [6]:
def limpar_str(txt):
    model = re.compile('[^a-zA-Z0-9 ]')
    result = normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')
    return model.sub(' ', result)

In [10]:
def calcula_tf(documento, termo):
    return len(list(filter((lambda x: x.lower() == termo), documento)))

In [11]:
def indices_invertidos(data):
    indices = defaultdict(set)
    for i, row in data.iterrows():
        titulo = (word.lower() for word in (nl.word_tokenize(row['titulo'])))
        subTitulo = (word.lower() for word in (nl.word_tokenize(row['conteudo'])))
        conteudo = (word.lower() for word in (nl.word_tokenize(row['subTitulo'])))
        
        noticia = list(titulo) + list(subTitulo) + list(conteudo)
        
        for palavra in noticia:
            if(palavra in indices):
                temp = indices[palavra]
                if(row["idNoticia"] not in temp):
                    temp[row["idNoticia"]] = calcula_tf(noticia, palavra)
                    indices[palavra] = temp
            else:
                indices[palavra] = {row["idNoticia"]: calcula_tf(noticia, palavra)}
    return(indices)

indices_invertidos = indices_invertidos(df)

In [12]:
def idf(M, k): return  math.log((M + 1) / k)